In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup as bs
import requests
from csv import writer
import pandas as pd
from lxml import html

In [3]:
url='https://www.bookdepository.com/bestsellers'

In [4]:
resp=requests.get(url)
resp

<Response [200]>

In [7]:
Page = 1
all_Format = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    formats_on_page = soup.select("div.item-info p.format")
    for product_format in formats_on_page:
        all_Format.append(product_format.get_text())
    Page = Page + 1
formats_series = pd.Series(all_Format)
formats_series.value_counts() #Gives count of Formats

Paperback              714
Hardback               295
Board book              31
Game                     2
Cards                    2
Book                     1
Toy                      1
CD-Audio                 1
Novelty book             1
Mixed media product      1
Sheet music              1
dtype: int64

In [8]:
formats_series

0        Hardback
1       Paperback
2       Paperback
3       Paperback
4            Game
          ...    
1045    Paperback
1046     Hardback
1047    Paperback
1048    Paperback
1049    Paperback
Length: 1050, dtype: object

In [9]:
formats_series.value_counts(normalize=True)

Paperback              0.680000
Hardback               0.280952
Board book             0.029524
Game                   0.001905
Cards                  0.001905
Book                   0.000952
Toy                    0.000952
CD-Audio               0.000952
Novelty book           0.000952
Mixed media product    0.000952
Sheet music            0.000952
dtype: float64

In [10]:
Page = 1
all_years = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    years_on_page = soup.select("div.item-info p.published")
    for product_year in years_on_page:
        a = product_year.get_text()
        #print(a)
        py= a.split(" ")[2]
        #print(py)
        all_years.append(py)
        #PY = product_year.get_text().split(" ")[2]
    page = Page + 1
years_series = pd.Series(all_years)
years_series.value_counts()  #Gives count of Years

2022    393
2021     91
2020     79
2018     69
2017     57
2019     51
2016     42
2011     40
2023     37
2015     37
2014     27
2012     21
2013     17
2007     11
2010     10
2003      8
2005      8
2009      8
2008      7
2000      6
2006      6
1995      6
1997      5
2001      3
1998      2
1996      2
1999      2
1987      2
2002      1
2004      1
dtype: int64

In [11]:
years_df = years_series.value_counts().to_frame().reset_index()
years_df.rename(columns={"index":"Year", 0:"Published books"}, inplace=True)
print(years_df)

    Year  Published books
0   2022              393
1   2021               91
2   2020               79
3   2018               69
4   2017               57
5   2019               51
6   2016               42
7   2011               40
8   2023               37
9   2015               37
10  2014               27
11  2012               21
12  2013               17
13  2007               11
14  2010               10
15  2003                8
16  2005                8
17  2009                8
18  2008                7
19  2000                6
20  2006                6
21  1995                6
22  1997                5
23  2001                3
24  1998                2
25  1996                2
26  1999                2
27  1987                2
28  2002                1
29  2004                1


In [15]:
Page = 1
all_titles = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    titles_on_page = soup.select("div.item-info h3.title")
    for product_title in titles_on_page:
        a = product_title.get_text(strip=True)
        #print(a)
        all_titles.append(a)
    Page = Page + 1
title_series = pd.Series(all_titles)
title_series.value_counts()    # Give count of Titles

The Little Book of Common Sense Investing    3
Small Things Like These                      3
Never Finished                               3
Never Split the Difference                   3
The Secret History                           3
                                            ..
Finding the Mother Tree                      1
The Myth of Sisyphus                         1
Minecraft Annual 2023                        1
SPACEBOY                                     1
Jokes for Funny Kids: 8 Year Olds            1
Length: 990, dtype: int64

In [16]:
Page = 1
books_Price = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    a = soup.find("span", class_="sale-price").get_text().split("€")[0].replace(",", ".")
    #current_price = a.split("€")[0].replace(",", ".")
    print(a)
        
    Page = Page + 1

16.80 
16.80 
10.18 
6.96 
8.72 
6.36 
8.22 
15.57 
11.92 
15.09 
12.51 
50.59 
11.24 
8.81 
12.72 
12.23 
8.57 
35.23 
39.97 
12.82 
56.22 
10.34 
18.76 
7.75 
6.86 
8.35 
3.47 
41.17 
9.66 
7.32 
18.10 
17.06 
9.57 
9.96 
8.12 


In [16]:
# Merging all 4 featues into one program in dictionary and list

page = 1
bestseller_books = []
bestseller_book = {}
for page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    div = soup.find("div", class_="tab-wrap module type-book grid tab--2 tab-active")
    for book in soup.find_all('div', class_="book-item"):  
        bestseller_book["title"] = book.find("h3", class_="title").get_text(strip=True)
        #print(bestseller_book["title"])
        bestseller_book["format"] = book.find("p", class_="format").get_text()
        #print(bestseller_book["format"])
        try:
            bestseller_book["year"] = book.find("p", class_="published").get_text()[-4:]
        except AttributeError:
            bestseller_book["year"] = ""
        #print(bestseller_book["year"])
        try:
            bestseller_book["price"] = book.find("span", class_="sale-price").get_text().split("€")[0].replace(",", ".")
        except AttributeError:
                  bestseller_book["price"] = ""
    
        #print(bestseller_book)
        #print(count)
        bestseller_books.append(bestseller_book.copy())
        page = page + 1
        
print(bestseller_books)

[{'title': 'The Little Book of Common Sense Investing', 'format': 'Hardback', 'year': '2017', 'price': '16.80 '}, {'title': "It Ends With Us: The most heartbreaking novel you'll ever read", 'format': 'Paperback', 'year': '2016', 'price': '7.37 '}, {'title': 'Before the Coffee Gets Cold', 'format': 'Paperback', 'year': '2019', 'price': '9.47 '}, {'title': 'Verity', 'format': 'Paperback', 'year': '2022', 'price': '6.58 '}, {'title': 'Undaunted: Stalingrad', 'format': 'Game', 'year': '2022', 'price': ''}, {'title': 'Me vs Brain', 'format': 'Hardback', 'year': '2023', 'price': '21.48 '}, {'title': 'Atomic Habits', 'format': 'Paperback', 'year': '2018', 'price': '15.27 '}, {'title': 'Four Thousand Weeks', 'format': 'Paperback', 'year': '2022', 'price': '9.86 '}, {'title': 'The Tools', 'format': 'Paperback', 'year': '2014', 'price': '17.18 '}, {'title': 'Seven Husbands of Evelyn Hugo', 'format': 'Paperback', 'year': '2021', 'price': '7.43 '}, {'title': 'The Midnight Library', 'format': 'Pape

In [18]:
# Dictionary with count
count = 0
for book in bestseller_books:
    count +=1
    print(book)
    print(count)

    

{'title': 'The Little Book of Common Sense Investing', 'format': 'Hardback', 'year': '2017', 'price': '16.80 '}
1
{'title': "It Ends With Us: The most heartbreaking novel you'll ever read", 'format': 'Paperback', 'year': '2016', 'price': '7.37 '}
2
{'title': 'Before the Coffee Gets Cold', 'format': 'Paperback', 'year': '2019', 'price': '9.47 '}
3
{'title': 'Verity', 'format': 'Paperback', 'year': '2022', 'price': '6.58 '}
4
{'title': 'Undaunted: Stalingrad', 'format': 'Game', 'year': '2022', 'price': ''}
5
{'title': 'Me vs Brain', 'format': 'Hardback', 'year': '2023', 'price': '21.48 '}
6
{'title': 'Atomic Habits', 'format': 'Paperback', 'year': '2018', 'price': '15.27 '}
7
{'title': 'Four Thousand Weeks', 'format': 'Paperback', 'year': '2022', 'price': '9.86 '}
8
{'title': 'The Tools', 'format': 'Paperback', 'year': '2014', 'price': '17.18 '}
9
{'title': 'Seven Husbands of Evelyn Hugo', 'format': 'Paperback', 'year': '2021', 'price': '7.43 '}
10
{'title': 'The Midnight Library', 'form

In [19]:
# Converted into DataFrame
books_df = pd.DataFrame(bestseller_books)
books_df

,title,format,year,price
0,The Little Book of Common Sense Investing,Hardback,2017,16.80
1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016,7.37
2,Before the Coffee Gets Cold,Paperback,2019,9.47
3,Verity,Paperback,2022,6.58
4,Undaunted: Stalingrad,Game,2022,
...,...,...,...,...
1045,Tribe of Mentors,Paperback,2017,17.41
1046,The Art & Science of Foodpairing,Hardback,2020,
1047,Bad Dreams and Other Stories,Paperback,2018,11.72
1048,Dreams for our Daughters,Paperback,2022,7.13
